In [14]:
from datasets import load_dataset
import datasets
import pandas as pd
import os

TRAIN_FILE = './data/train_set_v2.csv'
#EXTERNAL_FILE = './data/external-data.csv'
TEST_FILE = './data/test_set_v2.csv'
SPLIT_TRAIN_IN_VAL = False

modelo = './ATE/expreriments/mdeberta-v3-base-huggingface-more-training'
use_auth_token = None
MAX_LEN = 512
experiment_name = 'mdeberta-v3-base-huggingface-more-training-portuguese-data'
model_dir ='./ATE/expreriments/' + experiment_name + '/'

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

train_df = pd.read_csv(TRAIN_FILE, sep=';')
#external_df = pd.read_csv(EXTERNAL_FILE, sep=';')
test_df = pd.read_csv(TEST_FILE, sep=';')

ate_train_data = train_df.groupby('review').agg(list).reset_index()
ate_test_data = test_df.groupby('review').agg(list).reset_index()

In [2]:
uso_no_colab = False
if uso_no_colab:
  !pip install transformers datasets seqeval
  !pip install wandb

%env WANDB_PROJECT=ABSAPT22_ATE
%env WANDB_WATCH=all
%env WANDB_NOTEBOOK_NAME=experiment_name    
    
import wandb
wandb.login()

env: WANDB_PROJECT=ABSAPT22_ATE
env: WANDB_WATCH=all
env: WANDB_NOTEBOOK_NAME=experiment_name


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find experiment_name.
wandb: Currently logged in as: eduagarcia (use `wandb login --relogin` to force relogin)


True

In [3]:
if SPLIT_TRAIN_IN_VAL:
    from sklearn.model_selection import train_test_split
    ate_train_data, ate_dev_data = train_test_split(ate_train_data, test_size=0.2)
else:
    ate_dev_data = ate_test_data.copy()

In [4]:
def is_span_a_subset(span, aspect_span):
    if span[0] >= aspect_span[1]:
        return False
    elif span[1] < aspect_span[0]:
        return False
    else:
        return True

In [5]:
import nltk
nltk.download('punkt')
from nltk.tokenize.treebank import TreebankWordTokenizer   
def convert_to_bio(df):
    data = []
    for i, row in df.iterrows():
        #tokens = tokenize.word_tokenize(row['review'], language='portuguese')
        aspects_span = [[i, j, p, 0] for i, j, p in zip(row['start_position'], row['end_position'], row['polarity'])]
        tokens = []
        ner_tags = []
        span_generator = TreebankWordTokenizer().span_tokenize(row['review'])
        for span in span_generator:
            tokens.append(row['review'][span[0]:span[1]])
            is_aspect = False
            aspect_data = None
            for aspect_span in aspects_span:
                if is_span_a_subset(span, aspect_span):
                    is_aspect = True
                    aspect_data = aspect_span
            if is_aspect:
                label = 'ASPECT'
                
                #polarity_id = int(aspect_data[2])
                #if polarity_id == 1:
                #    label = 'POSITIVE'
                #elif polarity_id == 0:
                #    label = 'NEUTRAL'
                #else:
                #    label = 'NEGATIVE'
                
                if aspect_data[3] == 0:
                    ner_tags.append('B-'+label)
                    aspect_data[3] = aspect_data[3] + 1
                else:
                    ner_tags.append('I-'+label)
            else:
                ner_tags.append('O')
        data.append({'id': i, 'tokens': tokens, 'ner_tags': ner_tags})
    return data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
from datasets import Dataset
data = {
    'train': Dataset.from_pandas(pd.DataFrame(convert_to_bio(ate_train_data))),
    'validation':  Dataset.from_pandas(pd.DataFrame(convert_to_bio(ate_dev_data))),
    'test':  Dataset.from_pandas(pd.DataFrame(convert_to_bio(ate_test_data)))
}


dataset = datasets.DatasetDict(data)

In [8]:
sep = " "
if not os.path.exists('./ATE/data/aspect/'):
    os.makedirs('./ATE/data/aspect/')
for data_type in dataset.keys():
    with open(f'./ATE/data/aspect/{data_type}.conll', 'w') as f:
        for tokens, tags in zip(dataset[data_type]['tokens'], dataset[data_type]['ner_tags']):
            for token, tag in zip(tokens, tags):
                f.write(str(token)+sep+tag+'\n')
            f.write('\n')

In [9]:
label_list = list(set(tag for doc in dataset['train']['ner_tags'] for tag in doc))
label_list

['B-ASPECT', 'I-ASPECT', 'O']

In [10]:
label_list = ['B-ASPECT', 'O', 'I-ASPECT']

In [11]:
features = datasets.Features(
    {
        'id': datasets.Value('int32'),
        'tokens': datasets.Sequence(datasets.Value('string')),
        'ner_tags': datasets.Sequence(
            datasets.features.ClassLabel(names=label_list)
        )
    }
)

dataset = dataset.map(features.encode_example, features=features)

100%|██████████| 170/170 [00:00<00:00, 2052.65ex/s]


In [15]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(modelo, use_auth_token=use_auth_token)

In [16]:
tokenizer.__class__

transformers.models.deberta_v2.tokenization_deberta_v2.DebertaV2Tokenizer

In [17]:
tokenized_inputs = tokenizer(dataset['train']["tokens"], truncation=True, is_split_into_words=True, max_length=512)

In [18]:
tokenized_inputs['input_ids'][0]

[1,
 260,
 265,
 2725,
 1881,
 14135,
 260,
 262,
 520,
 5551,
 260,
 263,
 29287,
 271,
 7186,
 376,
 12299,
 269,
 260,
 362,
 674,
 12688,
 58522,
 4473,
 6544,
 260,
 262,
 1635,
 427,
 266,
 81385,
 261,
 140703,
 23852,
 264,
 270,
 29924,
 264,
 530,
 47068,
 261,
 260,
 14338,
 260,
 362,
 14488,
 270,
 616,
 94878,
 260,
 262,
 260,
 362,
 260,
 269,
 5526,
 270,
 327,
 43814,
 261,
 323,
 299,
 585,
 43307,
 270,
 5471,
 7461,
 260,
 266,
 215299,
 2713,
 333,
 263,
 270,
 8013,
 11024,
 295,
 47603,
 270,
 60034,
 5236,
 260,
 362,
 1059,
 751,
 260,
 262,
 260,
 263,
 260,
 27063,
 260,
 362,
 1913,
 3886,
 15070,
 338,
 270,
 333,
 263,
 261,
 323,
 3678,
 787,
 18241,
 351,
 47603,
 270,
 60034,
 5236,
 260,
 3529,
 16250,
 1665,
 2352,
 338,
 260,
 262,
 674,
 260,
 164900,
 270,
 260,
 32746,
 263,
 725,
 2228,
 93197,
 652,
 1209,
 21487,
 450,
 479,
 260,
 262,
 376,
 2096,
 1356,
 124643,
 1519,
 270,
 9518,
 4931,
 652,
 1209,
 431,
 479,
 260,
 261,
 2]

In [19]:
tokenizer(dataset['train']["tokens"][0][0], add_special_tokens=False, truncation=True, is_split_into_words=True, max_length=512)

{'input_ids': [260, 265], 'token_type_ids': [0, 0], 'attention_mask': [1, 1]}

In [20]:
#função que ajusta os labels para o tamanho dos textos após tokenização do BERT
#necessário pois palavras podem ser subdivididas com ##
def tokenize_and_align_labels(dataset_unaligned, label_all_tokens = False):
    tokenized_inputs = tokenizer(dataset_unaligned["tokens"], truncation=True, is_split_into_words=True, max_length=512)
    #print(tokenized_inputs)
    labels = []
    for i, label in enumerate(dataset_unaligned[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # None é o valor para tokens especiais -> -100 para ignorar na função de custo
            if word_idx is None: #special tokens
                label_ids.append(-100)
            #palavra nova
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            #label_all_tokens = True -> mesma tag para todos os subtokens
            #label_all_tokens = False -> apenas primeiro subtoken ganha tag
            else: #subpalavra
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
#função que ajusta os labels para o tamanho dos textos após tokenização do BERT
#necessário pois palavras podem ser subdivididas com ##
def tokenize_and_align_labels_python_tokenizer(dataset_unaligned, label_all_tokens = False):    
    tokenized_inputs = tokenizer(dataset_unaligned["tokens"], truncation=True, is_split_into_words=True, max_length=512)
    special_tokens = [tokenized_inputs['input_ids'][0][0], tokenized_inputs['input_ids'][0][-1]]
    
    word_ids_batch = []
    for tokens in dataset_unaligned["tokens"]:
        word_ids_batch.append([None])
        tokens_len = 2
        for word_idx, token in enumerate(tokens):
            token_processed = tokenizer(token, add_special_tokens=False, truncation=True, is_split_into_words=True, max_length=512)
            for i in range(len(token_processed['input_ids'])):
                tokens_len += 1
                if tokens_len <= 512:
                    word_ids_batch[-1].append(word_idx)
        word_ids_batch[-1].append(None)
    #print(tokenized_inputs)
    labels = []
    for i, label in enumerate(dataset_unaligned[f"ner_tags"]):
        word_ids = word_ids_batch[i]
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # None é o valor para tokens especiais -> -100 para ignorar na função de custo
            if word_idx is None: #special tokens
                label_ids.append(-100)
            #palavra nova
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            #label_all_tokens = True -> mesma tag para todos os subtokens
            #label_all_tokens = False -> apenas primeiro subtoken ganha tag
            else: #subpalavra
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [22]:
tokenized_datasets = dataset.map(tokenize_and_align_labels_python_tokenizer, batched=True)
tokenized_datasets

100%|██████████| 1/1 [00:01<00:00,  1.12s/ba]


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 677
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 170
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 170
    })
})

In [23]:
from datasets import load_metric
metric = load_metric("seqeval")

In [24]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [25]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [26]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(modelo, num_labels=len(label_list), use_auth_token=use_auth_token)

In [27]:
per_device_batch_size = 8
gradient_accumulation_steps = 1

total_steps_epoch = len(dataset['train']) // (per_device_batch_size * gradient_accumulation_steps)

learning_rate = 4e-5 
num_train_epochs = round(50000/total_steps_epoch)  
weight_decay = 0.01
warmup_ratio=0.1                         #  primeiros 10% --> Artigo Souza 2019

save_total_limit = 3
logging_steps = total_steps_epoch#(num_train_epochs*total_steps_epoch) // 20 # 20x por treinamento
eval_steps = logging_steps
evaluation_strategy = 'steps'
logging_strategy = 'steps'
save_strategy = 'steps'
save_steps = logging_steps
load_best_model_at_end = True

fp16 = False

# folders

folder_model = 'e' + str(num_train_epochs) + '_lr' + str(learning_rate)
output_dir = model_dir + 'results'
logging_dir = model_dir + 'results'
# get best model through a metric
metric_for_best_model = 'eval_f1'
if metric_for_best_model == 'eval_f1':
    greater_is_better = True
elif metric_for_best_model == 'eval_loss':
    greater_is_better = False  

args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_batch_size,
    per_device_eval_batch_size=per_device_batch_size*2,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    warmup_ratio=warmup_ratio,
    save_total_limit=save_total_limit,
    logging_steps = logging_steps,
    eval_steps = logging_steps,
    load_best_model_at_end = load_best_model_at_end,
    metric_for_best_model = metric_for_best_model,
    greater_is_better = greater_is_better,
    gradient_checkpointing = False,
    do_train = True,
    do_eval = True,
    do_predict = True,
    evaluation_strategy = evaluation_strategy,
    logging_dir=logging_dir, 
    logging_strategy = logging_strategy,
    save_strategy = save_strategy,
    save_steps = save_steps,
    fp16 = fp16,
    push_to_hub=False,
)

In [28]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [29]:
from transformers.trainer_callback import EarlyStoppingCallback

# wait early_stopping_patience x eval_steps before to stop the training in order to get a better model
early_stopping_patience = save_total_limit

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)],
)

In [30]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 677
  Num Epochs = 595
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 50575
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
84,0.019500,0.046801,0.799712,0.918874,0.855162,0.984352
168,0.018100,0.048890,0.814371,0.900662,0.855346,0.984745
252,0.017000,0.050844,0.814371,0.900662,0.855346,0.984666
336,0.015100,0.053096,0.813253,0.894040,0.851735,0.984430
420,0.014800,0.055378,0.813740,0.882450,0.846704,0.983958


The following columns in the evaluation set  don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 170
  Batch size = 16
Saving model checkpoint to ./ATE/expreriments/mdeberta-v3-base-huggingface-more-training-portuguese-data/results/checkpoint-84
Configuration saved in ./ATE/expreriments/mdeberta-v3-base-huggingface-more-training-portuguese-data/results/checkpoint-84/config.json
Model weights saved in ./ATE/expreriments/mdeberta-v3-base-huggingface-more-training-portuguese-data/results/checkpoint-84/pytorch_model.bin
tokenizer config file saved in ./ATE/expreriments/mdeberta-v3-base-huggingface-more-training-portuguese-data/results/checkpoint-84/tokenizer_config.json
Special tokens file saved in ./ATE/expreriments/mdeberta-v3-base-huggingface-mo

TrainOutput(global_step=420, training_loss=0.016910333292824882, metrics={'train_runtime': 150.2903, 'train_samples_per_second': 2680.247, 'train_steps_per_second': 336.515, 'total_flos': 323346657299886.0, 'train_loss': 0.016910333292824882, 'epoch': 4.94})

In [31]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 170
  Batch size = 16


{'eval_loss': 0.04889018088579178,
 'eval_precision': 0.8143712574850299,
 'eval_recall': 0.9006622516556292,
 'eval_f1': 0.8553459119496857,
 'eval_accuracy': 0.9847448297554454,
 'eval_runtime': 0.7361,
 'eval_samples_per_second': 230.961,
 'eval_steps_per_second': 14.945,
 'epoch': 4.94}

In [32]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 170
  Batch size = 16


{'ASPECT': {'precision': 0.8143712574850299,
  'recall': 0.9006622516556292,
  'f1': 0.8553459119496857,
  'number': 604},
 'overall_precision': 0.8143712574850299,
 'overall_recall': 0.9006622516556292,
 'overall_f1': 0.8553459119496857,
 'overall_accuracy': 0.9847448297554454}

In [33]:
trainer.save_model(model_dir)

Saving model checkpoint to ./ATE/expreriments/mdeberta-v3-base-huggingface-more-training-portuguese-data/
Configuration saved in ./ATE/expreriments/mdeberta-v3-base-huggingface-more-training-portuguese-data/config.json
Model weights saved in ./ATE/expreriments/mdeberta-v3-base-huggingface-more-training-portuguese-data/pytorch_model.bin
tokenizer config file saved in ./ATE/expreriments/mdeberta-v3-base-huggingface-more-training-portuguese-data/tokenizer_config.json
Special tokens file saved in ./ATE/expreriments/mdeberta-v3-base-huggingface-more-training-portuguese-data/special_tokens_map.json
added tokens file saved in ./ATE/expreriments/mdeberta-v3-base-huggingface-more-training-portuguese-data/added_tokens.json


In [34]:
import pandas as pd
resultado_eval = pd.DataFrame(results)
resultado_eval.to_csv(model_dir+'metricas_validation.csv')
resultado_eval

,ASPECT,overall_precision,overall_recall,overall_f1,overall_accuracy
f1,0.855346,0.814371,0.900662,0.855346,0.984745
number,604.000000,0.814371,0.900662,0.855346,0.984745
precision,0.814371,0.814371,0.900662,0.855346,0.984745
recall,0.900662,0.814371,0.900662,0.855346,0.984745


In [35]:
raw_predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(raw_predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 170
  Batch size = 16


{'ASPECT': {'precision': 0.8143712574850299,
  'recall': 0.9006622516556292,
  'f1': 0.8553459119496857,
  'number': 604},
 'overall_precision': 0.8143712574850299,
 'overall_recall': 0.9006622516556292,
 'overall_f1': 0.8553459119496857,
 'overall_accuracy': 0.9847448297554454}

In [36]:
resultado_teste = pd.DataFrame(results)
resultado_teste.to_csv(model_dir+'metricas_test.csv')
resultado_teste

,ASPECT,overall_precision,overall_recall,overall_f1,overall_accuracy
f1,0.855346,0.814371,0.900662,0.855346,0.984745
number,604.000000,0.814371,0.900662,0.855346,0.984745
precision,0.814371,0.814371,0.900662,0.855346,0.984745
recall,0.900662,0.814371,0.900662,0.855346,0.984745


In [37]:
o_index = label_list.index('O')
preds_index = np.asarray([i for i in range(len(label_list)) if i != o_index])

y_pred = []
y_true = []

for sentence_raw_prediction, sentence_labels in zip(raw_predictions, true_labels):
    for raw_prediction, true_label in zip(sentence_raw_prediction, sentence_labels):
        if true_label.startswith('B'):
            best_pred_idx = np.argmax(raw_prediction[preds_index])
            best_pred = preds_index[best_pred_idx]
            pred_label = label_list[best_pred][2:]
            true_label = true_label[2:]
            y_pred.append(pred_label)
            y_true.append(true_label)

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

      ASPECT       1.00      1.00      1.00       604

    accuracy                           1.00       604
   macro avg       1.00      1.00      1.00       604
weighted avg       1.00      1.00      1.00       604

